In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import datetime as dt
import pandas as pd
import numpy as np

from apidata import (
    quandl_stock_data,
    optimize_portfolio,
    closing_prices,
)

In [ ]:
stock_selection = [
    "AMZN",
    "PFE",
    "AMD",
    "HAL",
    "PG",
    "COF",    
]

stock_selection

In [ ]:
portfolio = optimize_portfolio(assets=stock_selection)

In [ ]:
def backtest_portfolio_performance(pfolio):
    exclude = ["Return", "Sharpe", "Variance"]
    assets = [(a,wt) for a, wt in pfolio.items() if a not in exclude]
    
    # Initialize Portfolio Back-Test Performance DataFrame
    back_test = closing_prices(assets[0][0]).set_index("Date")
    back_test = np.log(back_test / back_test.shift(1)).iloc[1:]
    back_test = back_test.apply(lambda x: x * assets[0][1])
    
    for allocation in assets[1:]:
        stock = allocation[0]
        weight = allocation[1]
        print(f'\nTicker: {stock} \tPortfolio Weight: {weight}')
        
        closing_data = closing_prices(stock).set_index("Date")
        pct_return = np.log(closing_data / closing_data.shift(1)).iloc[1:]
        pct_return = pct_return.apply(lambda x: x * weight)
        back_test = pd.merge(back_test, pct_return, on="Date", how="inner")
        print(pct_return.head())
        
    back_test["RTNp"] = back_test.sum(axis=1)
    print("\n[Historic] Portfolio Performance:\n", back_test.head())
    
    return back_test

In [ ]:
historic_perform = backtest_portfolio_performance(pfolio=portfolio)

In [ ]:
historic_perform.tail(10)

In [ ]:
historic_perform.describe()

In [ ]:
x_min = dt.date(2014, 1, 1)
x_max = dt.date(2018, 3, 27)

historic_perform["RTNp"].plot(alpha=0.75)
plt.grid(axis="y", linestyle="dashed")
plt.hlines(.02, x_min, x_max, color="red", linestyle="dashed")
plt.hlines(-.02, x_min, x_max, color="red", linestyle="dashed")
plt.show()

In [ ]:
RTNp = historic_perform["RTNp"]
RTNp.head(10)

In [ ]:
type(RTNp)

In [ ]:
RTNp.map(lambda x: x + 1)
RTNp.head()